In [1]:
df = spark.read.csv('s3://machine-learning-upgrad-bucket/Data_for_Quiz_Segment.csv', header=True, inferSchema=True)
df.printSchema()

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
2,application_1657104984405_0003,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- click: boolean (nullable = true)
 |-- C1: integer (nullable = true)
 |-- banner_pos: integer (nullable = true)
 |-- site_id: string (nullable = true)
 |-- site_domain: string (nullable = true)
 |-- site_category: string (nullable = true)
 |-- app_id: string (nullable = true)
 |-- app_domain: string (nullable = true)
 |-- app_category: string (nullable = true)
 |-- device_id: string (nullable = true)
 |-- device_ip: string (nullable = true)
 |-- device_model: string (nullable = true)
 |-- device_type: integer (nullable = true)
 |-- device_conn_type: integer (nullable = true)
 |-- C14: integer (nullable = true)
 |-- C15: integer (nullable = true)
 |-- C16: integer (nullable = true)
 |-- C17: integer (nullable = true)
 |-- C18: integer (nullable = true)
 |-- C19: integer (nullable = true)
 |-- C20: integer (nullable = true)
 |-- C21: integer (nullable = true)
 |-- month: integer (nullable = true)
 |-- dayofweek: integer (nullable = true)
 |-- day: integer (nullable = true)
 |-- 

In [2]:
#Same fucntion as earlier to draw a sample from the data set. 

from pyspark.sql.types import IntegerType
from math import floor
from pyspark.sql.functions import rand
from pyspark.sql.functions import col

def stratifiedSample(df, N, labelCol="y"):
    ctx = df.groupby(labelCol).count()
    ctx = ctx.withColumn('frac', col("count") / df.count())
    frac = ctx.select("y", "frac").rdd.collectAsMap()
    pos = int(floor(frac[1] * N))
    neg = int(floor(frac[0] * N))
    posDF = df.filter(col(labelCol) == 1).orderBy(rand()).limit(pos)
    negDF = df.filter(col(labelCol) == 0).orderBy(rand()).limit(neg)
    return posDF.unionAll(negDF)

df = df.withColumn("y", df["click"].cast(IntegerType()))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
xdf = stratifiedSample(df, 5_000_000)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
#seperate neumerical and catagorical columns into lists for ease of use

categorical = ['banner_pos', 'site_category', 'site_domain', 'device_type', 'app_domain', 'app_category',
               'device_model']
numerical = ['month', 'day', 'hour']

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
from pyspark.ml.feature import StringIndexer

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
# use the list created earlier to inpect the data. The * used brfore the list signifies that the list can be of any size. 
xdf.select(*categorical).limit(5).show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+-------------+-----------+-----------+----------+------------+------------+
|banner_pos|site_category|site_domain|device_type|app_domain|app_category|device_model|
+----------+-------------+-----------+-----------+----------+------------+------------+
|         0|     28905ebd|   f3845767|          1|  7801e8d9|    07d7df22|    8a4875bd|
|         1|     f028772b|   17d996e6|          1|  7801e8d9|    07d7df22|    8a4875bd|
|         0|     28905ebd|   f3845767|          1|  7801e8d9|    07d7df22|    be87996b|
|         0|     3e814130|   2296b5b3|          1|  7801e8d9|    07d7df22|    6e1e2240|
|         1|     f028772b|   7e091613|          1|  7801e8d9|    07d7df22|    e9b8d8d7|
+----------+-------------+-----------+-----------+----------+------------+------------+

In [7]:
# use the list notation to interate through string index for the columns in the list

stringCols = ['site_category', 'site_domain', 'app_domain', 'app_category', 'device_model']
newCols = []
for c in stringCols:
    #the putput columns will be marked bu "_x" for easy identification
    si = StringIndexer(inputCol=c, outputCol=c + '_ix')
    xdf = si.fit(xdf).transform(xdf)
    # adding the output columns with indexed data into a new list. 
    newCols.append(c + '_ix')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [8]:
# add the nuemerical type columns to the list. 
cCols = newCols + ['banner_pos', 'device_type']
cCols

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['site_category_ix', 'site_domain_ix', 'app_domain_ix', 'app_category_ix', 'device_model_ix', 'banner_pos', 'device_type']

In [9]:
#encolde all the columns in the list together
from pyspark.ml.feature import OneHotEncoderEstimator

ohe = OneHotEncoderEstimator(inputCols=cCols, outputCols=[c+'Enc' for c in cCols])
oh_encoder = ohe.fit(xdf)
xdf = oh_encoder.transform(xdf)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [10]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.classification import LogisticRegression

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [11]:
#create a feature vector using the vector assembler. 
trainCols = [c+'Enc' for c in cCols] + numerical
assembler = VectorAssembler(inputCols=trainCols, outputCol='features')
xdf = assembler.transform(xdf)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [12]:
#create the logistic regression object
lr = LogisticRegression(featuresCol='features', labelCol='y')
#Model fit it iver the data 
model = lr.fit(xdf)
#evlautate the data using the evaluate method. 
result = model.evaluate(xdf)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [13]:
#getting the result metrics 'accuracy'
result.accuracy

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

0.8773

In [14]:
#getting the result metrics 'Area under ROC'
result.areaUnderROC

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

0.8550420392156863

In [15]:
#getting the result metrics 'Reacll by label'. the first recall is for the label 0 and the second one is for the label 1
result.recallByLabel

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[0.9850588235294118, 0.26666666666666666]